# Experiments (Execution)

In [1]:
using CSV
using Dates
using DataFrames: DataFrame
using Random: MersenneTwister
using StatsBase: mean, std

include("../src/Simulation.jl")
using Main.Simulation: Param, Model, Strategy, C, POPULATION, PAYOFF, STRATEGY_MUTATION, RELATIONSHIP_MUTATION, interaction!, death!, birth!

include("../src/Network.jl")
using Main.Network: normalize_degree!, nv, weighted_cc, weighted_degree, normalize_degree!, normalize_weight!

In [2]:
function print_time()::Nothing
    println(Dates.format(now(), "YYYY-mm-dd HH:MM:SS"))
end

round4(x::Real) = round(x, digits = 4)

function run(model::Model, file_name::String)::Nothing
    print_time()

    open("$(file_name)_k999_τ10.csv", "w") do io
        write(io, "generation,N,cooperation_rate,wcc_C_mean,wcc_C_std,wcc_CD_mean,wcc_CD_std,wk_C_mean,wk_C_std,wk_CD_mean,wk_CD_std\n")

        for generation = 1:(model.param.generations)
            model.generation = generation
            model.payoff_vec .= 0.0

            interaction!(model)
            death!(model, model.param.rng)
            birth!(model, model.param.rng)

            normalize_degree!(model.weights, model.param.initial_k)
            normalize_weight!(model.weights, model.param.initial_k, model.param.initial_w)

            # Log
            C_index = (model.strategy_vec .== C)
            current_N = nv(model.weights)
            C_rate = round4(mean(C_index))

            wcc_CD_vec = weighted_cc(model.weights)
            wcc_C_vec = wcc_CD_vec[C_index]
            wcc_C_mean = round4(mean(wcc_C_vec))
            wcc_C_std = round4(std(wcc_C_vec))
            wcc_CD_mean = round4(mean(wcc_CD_vec))
            wcc_CD_std = round4(std(wcc_CD_vec))

            wk_CD_vec = weighted_degree(model.weights)
            wk_C_vec = wk_CD_vec[C_index]
            wk_C_mean = round4(mean(wk_C_vec))
            wk_C_std = round4(std(wk_C_vec))
            wk_CD_mean = round4(mean(wk_CD_vec))
            wk_CD_std = round4(std(wk_CD_vec))
            
            write(io, "$(generation),$(current_N),$(C_rate),$(wcc_C_mean),$(wcc_C_std),$(wcc_CD_mean),$(wcc_CD_std),$(wk_C_mean),$(wk_C_std),$(wk_CD_mean),$(wk_CD_std)\n")
        end
    end

    print_time()
end;

In [3]:
trial = 10
timesteps = 6000

# Common parameters
common_params = Dict{Symbol, Any}(
    :initial_N => 200,
    :initial_k => 199,
    :initial_T => 1.15,
    :S => -0.15,
    :initial_w => 0.2,
    :Δw => 0.05,
    :interaction_freqency => 1.0,
    :reproduction_rate => 0.05,
    :δ => 1.0,
    :initial_μ_s => 0.01,
    :initial_μ_r => 0.01,
    :generations => timesteps,
)

# Stable parameters
stable_params = copy(common_params)
stable_params[:β] = 0.0
stable_params[:sigma] = 0.0

# POPULATION variable parameters
population_params = copy(common_params)
population_params[:variability_mode] = POPULATION
population_params[:β] = 0.8
population_params[:sigma] = 180.0

# PAYOFF variable parameters
payoff_params = copy(common_params)
payoff_params[:variability_mode] = PAYOFF
payoff_params[:β] = 0.8
payoff_params[:sigma] = 0.5

# STRATEGY_MUTATION variable parameters
strategy_params = copy(common_params)
strategy_params[:variability_mode] = STRATEGY_MUTATION
strategy_params[:β] = 0.8
strategy_params[:sigma] = 0.01

# RELATIONSHIP_MUTATION variable parameters
relationship_params = copy(common_params)
relationship_params[:variability_mode] = RELATIONSHIP_MUTATION
relationship_params[:β] = 0.8
relationship_params[:sigma] = 0.01

# Paramベクトルの生成
# param_dic_vec = [stable_params, population_params, payoff_params, strategy_params, relationship_params]
param_dic_vec = [stable_params, population_params]
param_vec_vec = [[Param(; param..., rng = MersenneTwister(seed)) for seed in 1:trial] for param in param_dic_vec]

# Modelベクトルの生成
model_vec_vec = [[Model(param) for param in param_vec] for param_vec in param_vec_vec];

In [4]:
for seed in 1:10
    @time run(Model(Param(; stable_params..., rng = MersenneTwister(seed))), "experiments/output_stable_$(seed)")
    @time run(Model(Param(; population_params..., rng = MersenneTwister(seed))), "experiments/output_population_$(seed)")
    # @time run(Model(Param(; payoff_params..., rng = MersenneTwister(seed))), "experiments/output_payoff_$(seed)")
    # @time run(Model(Param(; strategy_params..., rng = MersenneTwister(seed))), "experiments/output_strategy_$(seed)")
    # @time run(Model(Param(; relationship_params..., rng = MersenneTwister(seed))), "experiments/output_relationship_$(seed)")
end

2024-02-08 17:30:50
2024-02-08 17:31:43
 52.896641 seconds (27.83 M allocations: 10.673 GiB, 0.74% gc time, 0.41% compilation time)
2024-02-08 17:31:43
2024-02-08 17:32:29
 46.067549 seconds (23.71 M allocations: 12.476 GiB, 1.44% gc time)
2024-02-08 17:32:29
2024-02-08 17:33:18
 49.317399 seconds (27.57 M allocations: 10.508 GiB, 0.78% gc time)
2024-02-08 17:33:18
2024-02-08 17:34:02
 44.336933 seconds (23.22 M allocations: 12.253 GiB, 1.60% gc time)
2024-02-08 17:34:02
2024-02-08 17:34:54
 51.645319 seconds (27.35 M allocations: 10.301 GiB, 0.78% gc time)
2024-02-08 17:34:54
2024-02-08 17:35:44
 49.863183 seconds (24.92 M allocations: 13.426 GiB, 1.59% gc time)
2024-02-08 17:35:44
2024-02-08 17:36:35
 50.807443 seconds (27.42 M allocations: 10.361 GiB, 0.81% gc time)
2024-02-08 17:36:35
2024-02-08 17:37:21
 46.077771 seconds (23.81 M allocations: 12.680 GiB, 1.48% gc time)
2024-02-08 17:37:21
2024-02-08 17:38:11
 49.858959 seconds (27.10 M allocations: 10.095 GiB, 0.75% gc time)
2024